In [21]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import pandas as pd
import time
import warnings
import numpy as np
import pyperclip
import random
import sys
import datetime

In [16]:
warnings.filterwarnings(action='ignore')

#크롬 웹 드라이버의 경로 설정
options = webdriver.ChromeOptions()
# options.add_argument("--headless")  # 브라우저 숨기기
driver = webdriver.Chrome('C:\chromedriver.exe', chrome_options=options)

# 네이버 로그인 페이지 접속
driver.get("https://nid.naver.com/nidlogin.login")

In [17]:
# 로그인 정보
login = {"id" : "chaton59", "pw" : "beauty5900"}

# 로그인 정보 입력 함수
def clipboard_input(user_xpath, user_input):
    temp_user_input = pyperclip.paste()

    pyperclip.copy(user_input)
    driver.find_element_by_xpath(user_xpath).click()
    ActionChains(driver).key_down(Keys.CONTROL).send_keys('v').key_up(Keys.CONTROL).perform()

    pyperclip.copy(temp_user_input)
    time.sleep(1)

# id, pw 입력 후 클릭
clipboard_input('//*[@id="id"]', login.get("id"))
clipboard_input('//*[@id="pw"]', login.get("pw"))
driver.find_element_by_xpath('//*[@id="log.login"]').click()

time.sleep(5)

In [18]:
# 결과 저장 경로
save_path = "C:\jupiter_workspace\zam_project\outputs"

# 카페 정보
cafe = {'name': '대구맘365','page_link': 'https://cafe.naver.com/dgmom365'}
cafe.update({"keywords" : ["기저귀"]})         # 검색 키워드

### 카페 게시글 링크 수집

In [40]:
for keyword in cafe.get("keywords"):
    ### 카페 주소 입력
    driver.get(cafe.get("page_link"))
    
    ### 키워드 검색
    clipboard_input('//*[@id="topLayerQueryInput"]', keyword)
    try :
        driver.find_element_by_xpath('//*[@id="cafe-search"]/form/button').click()   # 왼쪽에 검색창
    except:
        driver.find_element_by_xpath('//*[@id="info-search"]/form/button').click()   # 오른쪽에 검색창
    driver.implicitly_wait(0.5)
    driver.switch_to.frame('cafe_main')

    ### 키워드 수집 정보
    num_per_page = 15          # 페이지당 게시글 갯수(default: 15개)

    address_list=[]
    #address_df = pd.DataFrame(columns=['no'])
    page = 1
    
    for i in range(2):
        
        time.sleep( random.randint(0,5) )
        
        ### 현재 페이지의 html 불러오기
        r = driver.page_source
        page_html = BeautifulSoup(r, "html.parser")
        content = page_html.find("div", class_="article-board result-board m-tcol-c").find('tbody')
#         content = page_html.find_all("div", class_="article-board m-tcol-c")[1].find('tbody')
        body = content.find_all("tr")

        ### 게시글 정보 저장하기
        for x in body:
            temp_dict={}
            if x.find("div", class_="board-number") is not None:
                temp_dict['no'] = x.find("div", class_="board-number").text.strip()
                temp_dict['title'] = x.find("div", class_="board-list").text.strip().replace('  ','').replace('\n','')
                temp_dict['link'] = x.find('a').get('href')
                temp_dict['name'] = x.find("td", class_="td_name").find('a',class_='m-tcol-c').text.strip()
                temp_dict['date'] = x.find("td", class_="td_date").text.strip()
                temp_dict['view'] = x.find("td", class_="td_view").text.strip()
                address_list.append(temp_dict)
        print("(현재시각) "+str(datetime.datetime.now())+": "+ str(page) +"page done")

        ### 다음 페이지로 넘어가기
        page+=1
        driver.implicitly_wait(1)
        
        try:
            if page<=10:   # 1~10 : 페이지 번호 그대로
                page_xpath = str(page)
                driver.find_element_by_xpath('//*[@id="main-area"]/div[7]/a[' + page_xpath + ']').click()
            elif page == 11:   # 11 : 다음 버튼
                driver.find_element_by_xpath('//*[@id="main-area"]/div[7]/a[11]/span').click()
            elif page>11 and page%10!=1:   # 12~ : 페이지 번호 마지막 자리 + 1
                page_xpath = str(page-((page-1)//10)*10+1)
                driver.find_element_by_xpath('//*[@id="main-area"]/div[7]/a[' + page_xpath + ']').click()
            elif page%10 == 1:   # 21,31.. : 다음 버튼
                driver.find_element_by_xpath('//*[@id="main-area"]/div[7]/a[12]/span').click()
        except:
                print("(현재시각) "+str(datetime.datetime.now())+": done")

(현재시각) 2022-05-12 10:41:53.798953: 1page done
(현재시각) 2022-05-12 10:41:57.622437: 2page done


In [41]:
address_df = pd.DataFrame(address_list)
address_df['idx_no'] = range(1,len(address_df)+1)   # 조인할 키 값
address_df.to_pickle(save_path+"cafe_address_"+cafe.get("name")+"_"+keyword+".pkl")
print("(현재시각) "+str(datetime.datetime.now())+": done")

if len(set(address_df['no']))!=len(address_df) :
    print("게시글 번호에 중복 존재")
print("검색게시글수 : ", address_df.shape)

address_df.head()

(현재시각) 2022-05-12 10:42:00.507173: done
검색게시글수 :  (30, 7)


,no,title,link,name,date,view,idx_no
0,5840389,"[판매중]구찌 백( 정품)판매사진new개인거래(판매), 판매중, 가격 230,000...",/ca-fe/ArticleRead.nhn?clubid=24000254&page=1&...,옥포v달비채,09:41,133,1
1,5840223,자면서 응가ㅡㅡㅡ[6]new이런적은 처음인데요ㅡㅡ 방금 기저귀갈면서 보니.악!응가를...,/ca-fe/ArticleRead.nhn?clubid=24000254&page=1&...,월성새내기,05:02,327,2
2,5840003,성서에 요로감염 병원찾아요[6]new아기가 열은 안나는데 기저귀에 콧물처럼 이물질이...,/ca-fe/ArticleRead.nhn?clubid=24000254&page=1&...,이곡융,2022.05.11.,103,3
3,5839885,영아수당 신청 달바껴서 신청하면요~(+기저귀바우처)[6]new지난달 출산해서 조리원...,/ca-fe/ArticleRead.nhn?clubid=24000254&page=1&...,딩가v북구,2022.05.11.,111,4
4,5839744,맘님들이라면 어쩌시겠어요?[7]new오늘 아침에 아기 기저귀 갈다가 사타구니쪽이 볼...,/ca-fe/ArticleRead.nhn?clubid=24000254&page=1&...,혁신쑥쑥맘89,2022.05.11.,499,5


In [42]:
import pickle
from contextlib import suppress

for keyword in cafe.get("keywords"):
    df = pickle.load(open(save_path+"cafe_address_"+cafe.get("name")+"_"+keyword+".pkl", 'rb'))

    i=0
    contents_list = []   # 내용
    reply_list = []      # 댓글
    error_list = []      # 에러난 게시글

    while True:

        ### 수집 링크로 이동
        url = "https://cafe.naver.com"+df.loc[i,'link']
        idx_no = df.loc[i,'idx_no']    # 인덱스 번호
        driver.get(url)
        time.sleep( random.randint(2,5) )
        try:
            driver.switch_to.frame('cafe_main')
            time.sleep( random.randint(2,5) )
            r = driver.page_source
            page_soup = BeautifulSoup(r, "html.parser")
            content = page_soup.find('div', class_='ArticleContentBox')  

            ### 게시글 수집
            temp_dict={}
            temp_dict['idx_no'] = idx_no
            temp_dict['title'] = ""
            with suppress(AttributeError):   # 제목 없는 게시글
                temp_dict['title'] = content.find('h3',class_='title_text').text.strip()
            temp_dict['content'] = content.find("div", class_="article_viewer").text.strip()
            temp_dict['nick'] = content.find('div',class_='profile_info').find('a',class_='nickname').text.strip()
            temp_dict['date'] = content.find('div',class_='article_info').find('span',class_='date').text.strip()
            temp_dict['view'] = ""
            with suppress(AttributeError):
                temp_dict['view'] = content.find('div',class_='article_info').find('span',class_='count').text.strip()
            contents_list.append(temp_dict)

            ### 댓글 수집
            if content.find("div", class_="ReplyBox") is not None:   # 댓글 기능이 아예 없음  
                comment_num = content.find("div", class_="ReplyBox").find("a",class_="button_comment").find("strong").text
                if comment_num!='0':   # 댓글이 없음
                    comment = content.find("div", class_="CommentBox").find("ul",class_="comment_list").select("li")
                    
                    ### 댓글 구분
                    com_n=0    # 댓글
                    com_nn=0   # 대댓글
                    
                    for n in range(len(comment)):

                        if comment[n].get('class')==['CommentItem']:    # 댓글
                            com_n+=1; com_nn=0;
                            com_thread = str(com_n)+"-"+str(com_nn)
                            com_nn=1
                        elif comment[n].get('class')==['CommentItem', 'CommentItem--reply']:    # 대댓글
                            com_thread = str(com_n)+"-"+str(com_nn)
                            com_nn+=1

                        ### 댓글 내용 수집    
                        if comment[n].text.strip() != '삭제된 댓글입니다.':
                            com_nick = comment[n].find("a",class_="comment_nickname").text.strip()
                            com_date = comment[n].find("span",class_="comment_info_date").text.strip()
                            com_reply = comment[n].find("div",class_="comment_text_box").text.strip()
                            reply_list.append({'idx_no':idx_no, 'nick':com_nick, 'date':com_date, 'reply':com_reply, "thread":com_thread})
            i+=1

        except:
            i+=1
            ### 게시글을 볼 등급이 안됨
            if page_soup.find('strong', class_='emph') is not None:
                error_list.append({"error" :  page_soup.find('strong', class_='emph').text+"등급 필요", "url" : url})
                pass
            ### 에러 따로 확인
            else:
                error_list.append({"error" : "에러 확인 필요", "url" : url})
                pass

        ### 수집한 글 갯수만큼 반복
        if i == len(df):
            contents_df = pd.DataFrame(contents_list)
            contents_df.to_pickle("C:\jupiter_workspace\zam_project\outputs/cafe_"+cafe.get("name")+"_"+keyword+".pkl")
            reply_df = pd.DataFrame(reply_list)
            reply_df.to_pickle("C:\jupiter_workspace\zam_project\outputs/cafe_replies_"+cafe.get("name")+"_"+keyword+".pkl")
            print("(현재시각) "+str(datetime.datetime.now())+": done")
            break
    
# 크롬 종료 
driver.quit()

(현재시각) 2022-05-12 10:46:01.420271: done


In [49]:
comment

NameError: name 'comment' is not defined

In [45]:
# 수집한 데이터 : contents_df
print("수집 데이터 : ", contents_df.shape)
# 에러 난 게시글 : error_list
print("에러 게시글 수 : ", len(error_list))

수집 데이터 :  (0, 0)
에러 게시글 수 :  30
